# Sensitivity of gradiometers and magnetometers on sources in retinotopic labels

Turns out we cannot combine the grads and mags into one "conglomerate" sensitivity map: the scale of the signals is so different that just based on the forward model, the gradiometers have far higher sensitivity! To make a proper comparison, we must interrogate the CTFs.

However, as a first approximation as to how much signal each label can produce, relative to other labels, we can just use one sensor type. I predict that the results will depend only little on which is used (to be verified!)

In [1]:
%matplotlib qt

In [2]:
data_path = '/Users/cjb/projects/CFF_Retinotopy/scratch'
subject = '030_WAH'
sens_type = 'mag'
normalize_sens_map = True
plot3d_hemi = 'lh'
plot3d_reg = 'V3'
plot_x3d = False

In [3]:
from retinotopic_helpers import *
from mne import sensitivity_map, read_forward_solution
import matplotlib.pyplot as plt

In [4]:
if plot_x3d:
    from mayavi import mlab
    mlab.init_notebook()

In [5]:
subjects_dir = join(data_path, 'fs_subjects_dir')
fwd_fname = join(data_path, 'VS1-ico5-fwd.fif')

# Read the forward solutions with surface orientation
fwd = read_forward_solution(fwd_fname, surf_ori=True)
print("Leadfield size : %d x %d" % fwd['sol']['data'].shape)

Reading forward solution from /Users/cjb/projects/CFF_Retinotopy/scratch/VS1-ico5-fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (20484 sources, 306 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
    Converting to surface-based source orientations...
    Average patch normals will be employed in the rotation to the local surface coordinates....
[done]
Leadfield size : 306 x 61452


In [6]:
mode = 'fixed'  # 'free' is almost one-to-one with distance
sens_map = sensitivity_map(fwd, ch_type=sens_type, mode=mode,
                           norm=normalize_sens_map)
brain = sens_map.plot(time_label='Sensitivity', subjects_dir=subjects_dir,
                      clim=dict(lims=[0, 50, 100]), hemi=plot3d_hemi)

normalizer = 1 if normalize_sens_map else sens_map.data.max()

    102 out of 306 channels remain after picking
Updating smoothing matrix, be patient..
Smoothing matrix creation, step 1
Smoothing matrix creation, step 2
Smoothing matrix creation, step 3
Smoothing matrix creation, step 4
Smoothing matrix creation, step 5
Smoothing matrix creation, step 6
Smoothing matrix creation, step 7
Smoothing matrix creation, step 8
Smoothing matrix creation, step 9
Smoothing matrix creation, step 10
colormap: fmin=1.79e-02 fmid=3.95e-01 fmax=1.00e+00 transparent=1


/Users/cjb/miniconda_envs/oldqt/lib/python3.5/site-packages/mayavi-4.5.0-py3.5-macosx-10.6-x86_64.egg/mayavi/tools/camera.py:288: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if focalpoint is not None and not focalpoint == 'auto':


## Plot for all regions

In [7]:
labels = get_RM_labels(subject, subjects_dir=subjects_dir)

Reading labels from parcellation..
   read 18 labels from /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/label/lh.RM.V1.annot
[done]
Reading labels from parcellation..
   read 18 labels from /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/label/rh.RM.V1.annot
[done]
Reading labels from parcellation..
   read 18 labels from /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/label/lh.RM.V2.annot
[done]
Reading labels from parcellation..
   read 18 labels from /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/label/rh.RM.V2.annot
[done]
Reading labels from parcellation..
   read 18 labels from /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/label/lh.RM.V3.annot
[done]
Reading labels from parcellation..
   read 18 labels from /Users/cjb/projects/CFF_Retinotopy/scratch/fs_subjects_dir/030_WAH/label/rh.RM.V3.annot
[done]


In [8]:
fig = plt.figure(num=21,
                 figsize = (len(labels.keys())*polax_size, polax_size))
fig.clear()
for iax, reg in enumerate(labels.keys()):

    polax_bardata_reset()
    ax = plt.subplot(1, len(labels.keys()), iax + 1,
                    projection='polar')

    for hemi_bit, hemi in enumerate(('lh', 'rh')):
        for lab in labels[reg][hemi]:

            ecc_ind, ang_ind = get_ecc_ang_inds(lab.name)

            try:
#                 color = cm.hot(sens_map.in_label(lab).data.mean() / normalizer)
                val = sens_map.in_label(lab).data.mean() / normalizer
            except ValueError:
                val = None
#                 color = (1., 1., 1.)

                print('in_label fails at: {0:s}-{1:s}-{2:s}'.format(reg, hemi,
                                                                    lab.name))
#             else:
#                 print(reg, lab.name,
#                       '{:d}'.format(len(sens_map.in_label(lab).data)))
            
            polax_bardata_append(hemi_bit, ecc_ind, ang_ind, val)
#             c_plot.append(color)

            if hemi == plot3d_hemi and reg==plot3d_reg:
                cval = polax_get_colour(val, cmap=cm.hot,
                                        normalizer=normalizer)
                brain.add_label(lab, color=cval)

    polax_bardata_setcols(ax, cmap=cm.hot)
#     ax.set_theta_zero_location("N")
#     bars = ax.bar(th_plot, r_plot, width=w_plot, bottom=b_plot)
#     for val, bar in zip(val_plot, bars):
#         bar.set_facecolor(cm.hot(val) if not val is None else (1., 1., 1.))

in_label fails at: V2-lh-RM258-lh


/Users/cjb/miniconda_envs/oldqt/lib/python3.5/site-packages/mayavi-4.5.0-py3.5-macosx-10.6-x86_64.egg/mayavi/tools/camera.py:288: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if focalpoint is not None and not focalpoint == 'auto':
